In [1]:
import numpy as np

from tqdm import tqdm
from time import perf_counter
from attacker import SGA
from classifier import SGC, GCN

import utils

In [2]:
dataset = 'cora'
adj, x, y = utils.load_npz(dataset)
train_nodes, val_nodes, test_nodes = utils.train_val_test_split_tabular(y.shape[0], stratify=y, random_state=15)
degree = adj.sum(1).A1

In [3]:
model = SGC(adj, x, y, seed=None)
acc, train_time, f1_micro, f1_macro = model.train(train_nodes, val_nodes=val_nodes, verbose=False)
print(f'【Train】f1 micro {f1_micro:.2%}, f1_macro {f1_macro:.2%}, training time {train_time:.2%}')
f1_micro, f1_macro = model.test(test_nodes)
print(f'【Test】f1 micro {f1_micro:.2%}, f1_macro {f1_macro:.2%}')

【Train】f1 micro 83.94%, f1_macro 81.38%, training time 33.56%
【Test】f1 micro 84.76%, f1_macro 83.15%


In [4]:
W, b = model.get_weight()
attacker = SGA(adj, x, y, W, b)

In [5]:
edges = {}
t_all = {}
targets = utils.sample_targets(test_nodes, size=1000, seed=42)
for target, budget in tqdm(zip(targets, degree[targets]), desc="Attacking"):
    tb = perf_counter()
    attacker.attack(target, budget, direct_attack=True)
    te = perf_counter()
    edge = attacker.get_attack_edge()
    edges[target] = edge
    t_all[target] = te - tb


Attacking: 1000it [00:05, 171.82it/s]


In [6]:
print(f"Time used on average: {np.mean(list(t_all.values())):.4f}")

Time used on average: 0.0058


In [7]:
import os
import os.path as osp
if not osp.exists("../adv_edges/"):
    os.mkdir("../adv_edges")
attack_model = 'SGA'
np.savez(f'../adv_edges/{dataset}_{attack_model}', edges=edges, time=t_all)